In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
CREATE TABLE IF NOT EXISTS pyspark_cata.source.customers 
(
  id STRING,
  email STRING,
  city STRING,
  country STRING,
  ModifyDate TIMESTAMP
)

In [0]:
%sql
select * from pyspark_cata.source.customers

In [0]:
%sql
INSERT INTO pyspark_cata.source.customers
VALUES ('1','eklavya@databricks.com','patna','India', current_timestamp()),
('2','john.smith@databricks.com','jersey','USA',current_timestamp()),
('3','jane.doe@databricks.com','madrid','Spain',current_timestamp())

In [0]:
%sql
SELECT * FROM pyspark_cata.source.customers

In [0]:
from pyspark.sql.window import Window

df  = spark.read.table("pyspark_cata.source.customers")
df = df.withColumn("startDate",current_timestamp())\
    .withColumn("endDate",to_timestamp(lit("31-12-9999"),'dd-mm-yyyy'))\
        .withColumn("active",lit(True)).withColumn("id",col("id").cast("int"))

w = Window.partitionBy("id").orderBy(col("ModifyDate").desc())
df = df.withColumn("rn",row_number().over(w))
df = df.filter(col("rn")==1).drop("rn")
df.createOrReplaceTempView("src")
df.show()


In [0]:
from pyspark.sql.functions import *
if spark.catalog.tableExists('pyspark_cata.source.dimCustomers'):
    spark.sql("""
              MERGE INTO pyspark_cata.source.dimCustomers tgt
              USING src
              ON tgt.id = src.id AND tgt.active = True
              WHEN MATCHED AND tgt.city <> src.city 
              OR tgt.country<> src.country 
              OR tgt.email<> src.email 
              OR tgt.ModifyDate <> src.ModifyDate 
              THEN
              UPDATE 
              SET tgt.endDate = current_timestamp(),
              tgt.active = False
              """)
    
    spark.sql("""
              MERGE INTO pyspark_cata.source.dimCustomers tgt
              USING src
              ON tgt.id = src.id AND tgt.act ive = True

              WHEN NOT MATCHED THEN INSERT *
              """)

     
else:
    # df = spark.read.table("pyspark_cata.source.customers")
    # df = df.withColumn("startDate",current_timestamp())\
    #     .withColumn("endDate",to_timestamp(lit("31-12-9999"),'dd-mm-yyyy'))\
    #         .withColumn("acive",lit(True)).withColumn("id",col("id").cast("int"))

    #doing the same thing dome above in sql below using a temp view made on source df
     
    #spark.sql("select *,current_timestamp() as startDate,CAST("31-12-9999" as TIMESTAMP) as endDate
    #,True as active from srctmp")
    df.write.format("delta").mode("overwrite").saveAsTable("pyspark_cata.source.dimCustomers")

In [0]:
df = spark.read.table("pyspark_cata.source.dimCustomers")
df = df.drop("acive")
df.write.format("delta").mode("overwrite").option("overwriteSchema",True).saveAsTable("pyspark_cata.source.dimCustomers")

In [0]:
%sql
select * from pyspark_cata.source.dimCustomers
